In [15]:
from pathlib import Path

import asyncio
from contramate.llm import LLMVanillaClientFactory
from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.openai import OpenAIProvider
from pydantic import BaseModel
from pydantic_ai.run import AgentRunResult

In [16]:
ENV_FILE = Path().absolute().parent / ".envs" / "local.env"
print(f"Loading environment variables from: {ENV_FILE}")

Loading environment variables from: /Users/datapsycho/PythonProjects/AgentEngBootCamp/contramate/.envs/local.env


In [17]:
factory = LLMVanillaClientFactory.from_env_file(env_path=ENV_FILE)
client = factory.get_default_client(async_mode=True)
provider = OpenAIProvider(openai_client=client)
model = OpenAIChatModel(
        "gpt-4.1-mini",
        provider=provider,
    )

2025-10-20 19:38:15.172 | INFO     | contramate.llm.factory:get_default_client:81 - Creating vanilla async client for provider: openai


In [18]:
type(client)

openai.AsyncOpenAI

In [19]:
class QueryRewriteResponse(BaseModel):
    actual: str
    rewritten: str

In [30]:
SYSTEM_PROMPT = """
You are an expert at updating questions to make them ask for one thing only, more atomic, specific and easier to find the answer for.
You do this by filling in missing information in the question, with the extra information provided to you in previous answers. 

You respond with the updated question that has all information in it.
Only edit the question if needed. If the original question already is atomic, specific and easy to answer, you keep the original.
Do not ask for more information than the original question. Only rephrase the question to make it more complete.
You can also convert any known abbreviations to their full forms but also keep the abbreviations in parentheses next to the full form.
"""

In [21]:
agent = Agent(
        model=model,
        system_prompt=SYSTEM_PROMPT,
        output_type=QueryRewriteResponse,
    )


In [33]:
agent_response: AgentRunResult = await agent.run("What are the main benefits of AI?")

In [ ]:
agent_response.output

QueryRewriteResponse(actual='What are the main benefits of AI?', rewritten='What are the main benefits of artificial intelligence (AI)?')

In [ ]:
async def test_agent():
    """Simple test agent that says hello to AI"""
    factory = LLMVanillaClientFactory()
    client = factory.get_default_client(async_mode=True)
    provider = OpenAIProvider(openai_client=client)

    model = OpenAIChatModel(
        "gpt-4.1-mini",
        provider=provider,
    )

    agent = Agent(
        model=model,
        system_prompt="You are a friendly AI assistant. You always respond with properly formatted Markdown.",
    )
    result = await agent.run("What are the main benefits of cloud computing?")

    # Access the response data and metadata
    print("Agent response type:", type(result.output))
    print("\nAnswer:", result.output)
    print("\n--- Usage Information ---")

    # Access usage info
    usage = result.usage()
    print(f"Input tokens: {usage.input_tokens}")
    print(f"Output tokens: {usage.output_tokens}")
    print(f"Total tokens: {usage.total_tokens}")

    # Access the underlying model response for additional details
    if result.new_messages():
        last_response = result.new_messages()[-1]
        print(f"\nMessage timestamp: {last_response.timestamp}")
        print(f"Stop reason: {last_response.finish_reason}")
        print(f"Message ID: {last_response.provider_response_id}")
        print(f"Model name: {last_response.model_name}")
    

if __name__ == "__main__":
    asyncio.run(test_agent())

